In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

ttc = sns.load_dataset('titanic')
ttc.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
ttc.isnull().values.any()

True

-Hence there are null values present we need to treat those values

In [3]:
ttc.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [4]:
#out of the 891 entries, 688 values were not entered for the deck. Hence it is dropped in the analyis

ttc1= ttc.drop(['deck'], axis= 1)

In [5]:
##Modifications to dataset##

#survived and alive..........same feature..hence, alive is dropped

#embarked and embarked_town same feature.........hence embarked_town is dropped

#sibsb,parch and alone...........alone cance derived from other two.hence alone is dropped

#pclas and classs are the same.....class can be dropped

#adult_male is a subset of who..........hence adult_male is not considered in the analysis part
#using age and sex who can be foundout hence ,who is dropped

#but around 180 values of age is not filled, hence who is retained and age ,sex is dropped (hoping who is entered properly)

ttc2= ttc1.drop(['age','sex','adult_male','class','alone','embark_town','alive'], axis= 1)

ttc2.tail()

,survived,pclass,sibsp,parch,fare,embarked,who
886,0,2,0,0,13.00,S,man
887,1,1,0,0,30.00,S,woman
888,0,3,1,2,23.45,S,woman
889,1,1,0,0,30.00,C,man
890,0,3,0,0,7.75,Q,man


In [39]:
df=ttc2
df.head()

,survived,pclass,sibsp,parch,fare,embarked,who
0,0,3,1,0,7.2500,2,1
1,1,1,1,0,71.2833,0,2
2,1,3,0,0,7.9250,2,2
3,1,1,1,0,53.1000,2,2
4,0,3,0,0,8.0500,2,1


In [40]:
df.dropna(inplace=True)  # droping two rows where null values in embarked column

In [41]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

df['embarked']= le.fit_transform(df.embarked)
df['who']= le.fit_transform(df.who)

In [42]:
df.describe()

,survived,pclass,sibsp,parch,fare,embarked,who
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,2.311586,0.524184,0.382452,32.096681,1.535433,1.209224
std,0.486260,0.834700,1.103705,0.806761,49.697504,0.792088,0.593778
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,0.000000,7.895800,1.000000,1.000000
50%,0.000000,3.000000,0.000000,0.000000,14.454200,2.000000,1.000000
75%,1.000000,3.000000,1.000000,0.000000,31.000000,2.000000,2.000000
max,1.000000,3.000000,8.000000,6.000000,512.329200,2.000000,2.000000


In [53]:
xdata=df.drop('survived',axis=1)
#x
ydata=df.iloc[:,0:1]
#y

from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler(feature_range=(10,30))

xdata=pd.DataFrame(mms.fit_transform(xdata),columns=xdata.columns)


from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(xdata,ydata,test_size=0.3,stratify=ydata)

In [103]:
models=[LogisticRegression(),SVC(),DecisionTreeClassifier(),RandomForestClassifier(),KNeighborsClassifier()
       ,GaussianNB(), BernoulliNB()]

In [104]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [105]:
Emodels=[]
accuracy=[]
for model in models:
    model.fit(xtrain,ytrain)
    pred=model.predict(xtest)
    acc=accuracy_score(ytest,pred)*100
    accuracy.append(acc)
    Emodels.append(model)
    
    print(f"Metrics evaluation of model {model} :")
    print(f'Accuracy score :\n {acc}\n')
    print(f'Confusion matrix :\n {confusion_matrix(ytest,pred)}\n')
    print(f'Classification report :\n{classification_report(ytest,pred)}\n')
    print('_________________________________________________________\n')

Metrics evaluation of model LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) :
Accuracy score :
 73.40823970037454

Confusion matrix :
 [[143  22]
 [ 49  53]]

Classification report :
              precision    recall  f1-score   support

           0       0.74      0.87      0.80       165
           1       0.71      0.52      0.60       102

    accuracy                           0.73       267
   macro avg       0.73      0.69      0.70       267
weighted avg       0.73      0.73      0.72       267


_________________________________________________________

Metrics evaluation of model SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamm

In [106]:
accuracy_df=pd.DataFrame({'Model':Emodels,'Accuracy':accuracy})
accuracy_df

,Model,Accuracy
0,"LogisticRegression(C=1.0, class_weight=None, d...",73.408240
1,"SVC(C=1.0, break_ties=False, cache_size=200, c...",80.898876
2,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",81.273408
3,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",81.273408
4,"KNeighborsClassifier(algorithm='auto', leaf_si...",80.524345
5,"GaussianNB(priors=None, var_smoothing=1e-09)",74.157303
6,"BernoulliNB(alpha=1.0, binarize=0.0, class_pri...",61.797753
